# Data normalisation function

In [ ]:
def data_norm (data, method='log'):
    if method == 'log':
        tmp_log = data[data != 0]
        logOS = np.nanmedian(tmp_log)
        Data_log = np.log(data+logOS)
        Data_norm = Data_log
    
    if method == 'tic':
        Data_TIC=np.zeros((data.shape[0],data.shape[1]))
        for i in range (0,data.shape[0]):
            try:
                if data.values[i].all == 0:
                    Data_TIC[i] = 0
                else:
                    Data_TIC[i] = data.values[i]/np.sum(data.values[i])
            except:
                if data[i].all == 0:
                    Data_TIC[i] = 0
                else:
                    Data_TIC[i] = data[i]/np.sum(data[i])
        Data_norm = Data_TIC
        
    if method == 'minmax':
        Data_mm=np.zeros((data.shape[0],data.shape[1]))
        for i in range (0,data.shape[0]):
            try:
                if data.values[i].all == 0:
                    Data_mm[i] = 0
                else:
                    Data_mm[i] = (data.values[i]-np.min(data.values[i]))/np.max(data.values[i])
            except:
                if data.values[i].all == 0:
                    Data_mm[i] = 0
                else:
                    Data_mm[i] = (data[i]-np.min(data[i]))/np.max(data[i])
        Data_norm = Data_mm
        
    if method == 'snv':
        Data_snv=StandardScaler().fit_transform(data)
        Data_norm = Data_snv
    
    Data_norm = np.nan_to_num(Data_norm)
    return(Data_norm)

# Function for model training + CV

In [ ]:
def train_cv (data, method, cvsplit, label, group, weights = None, scoring = None):
    models = []

    if method == 'LR':
        if weights != None:
            models.append((LogisticRegression(solver='liblinear',class_weight = weights)))
        else:
            models.append((LogisticRegression(solver='liblinear')))

    if method == 'LDA':
        models.append((LinearDiscriminantAnalysis()))
    if method == 'KNN':
        models.append((KNeighborsClassifier()))
    if method == 'CART':
        models.append((DecisionTreeClassifier()))
    if method == 'NB':
        models.append((GaussianNB()))
    if method == 'SVM':
        if weights != None:
            models.append((SVC(probability=True, kernel = 'linear',
                                      decision_function_shape='ovo',class_weight = weights)))
        else:
            models.append((SVC(probability=True, kernel = 'linear',
                                      decision_function_shape='ovo')))
    if method == 'RF':
        numIterations = 10;
        num_trees = 150
        max_features = 10
        n_jobs = None
#         n_jobs = -1
        
        if weights != None:
            model = ((RandomForestClassifier(n_estimators= num_trees, max_features=max_features, 
                                             n_jobs = n_jobs,class_weight = weights)))
        else:
            model = ((RandomForestClassifier(n_estimators= num_trees, max_features=max_features, 
                                             n_jobs = n_jobs)))


    # Store results in a pandas dataframe
    results=pd.DataFrame(index=range(0,len(data)), columns=[method])
    n=1

    start_time = time.time()

    if cvsplit == 'loo':
        cvFold = logo.split(data,label,groups=group)
    if cvsplit == 'kfold':
        skf = StratifiedKFold(n_splits=10)
        cvFold = skf.split(data, group)

    for i, model in enumerate(models):
#         print(i)
        # only RF needs to be iterated as everything else deterministic
        if method == 'RF':
            RF_results=[]
            for n in range(0,numIterations):
                cvFold = logo.split(data,label,groups=group)
                ite = cross_val_predict(model, data, label, cv=cvFold)
                RF_results.append(ite)
                msg = "Method %s iteration %d \r" % (model, n)
                print(msg,end="")
#             results.iloc[:,i]=(RF_results/numIterations).sum(axis=1)
                results = RF_results
        else:
            results.iloc[:,i]=cross_val_predict(model, data, label, cv=cvFold)
            # Status message
            msg = "Method %s iteration %d \r" % (model, i)
            print(msg,end="")
    
    elapsed_time = time.time() - start_time
    print()
    print('that took '+str(elapsed_time)+' seconds')

    return results

# CV behaviour visualisation

In [ ]:
def multilabel_binarise (label):
    classes = np.unique(label)
    new_label = label
    for i in range (0,len(classes)):
        class_i = classes[i]
        new_label[new_label==class_i] = i
    return new_label


def plot_cv(cv, X, old_label, old_group, n_splits, lw=10):
    cmap_data = plt.cm.Paired
    cmap_cv = plt.cm.coolwarm
    fig, ax = plt.subplots()
    for ii, (tr, tt) in enumerate(cv.split(X,old_group)):
        # Fill in indices with the training/test groups
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(
            range(len(indices)),
            [ii + 0.5] * len(indices),
            c=indices,
            marker="_",
            lw=lw,
            cmap=cmap_cv,
            vmin=-0.2,
            vmax=1.2,
        )
    new_label = multilabel_binarise (old_label)
    new_group = multilabel_binarise (old_group)
    ax.scatter(
            range(len(X)), [ii + 1.5] * len(X), c=new_label, marker="_", lw=lw, cmap=cmap_data
        )
    ax.scatter(
        range(len(X)), [ii + 2.5] * len(X), c=new_group, marker="_", lw=lw, cmap=cmap_data
    )
    # Formatting
    yticklabels = list(range(n_splits)) + ["class", "group"]
    ax.set(
        yticks=np.arange(n_splits + 2) + 0.5,
        yticklabels=yticklabels,
        xlabel="Sample index",
        ylabel="CV iteration",
    )
    ax.set_title("{}".format(type(cv).__name__), fontsize=15)
    return ax

# function to calculate confusion matrix

In [ ]:
def calculate_cm (cv_results, label, perc=False):
    results=pd.DataFrame(index=range(0,len(label)), columns=['LOOCV results','labels'])
    results.iloc[:,0]=cv_results
    results.iloc[:,1]=label
    labelnames = set(label)
    labelnames = list(labelnames)
    dims = len(labelnames)
    
    #calculate diagonals for TPs & TNs
    TRUE = []
#     TN = []
    for i in range (0,dims):
        condition1=(results['LOOCV results']==1) & (results['labels'] == labelnames[i])
        dum=results[condition1]
        TRUE.append(len(dum))
    
    FALSE = []
    #calculate other elements
    for i in range (0,dims):
        condition2=(results['LOOCV results']==0) & (results['labels'] == labelnames[i])
        dum=results[condition2]
        FALSE.append(len(dum))
    c_matrix=np.zeros((dims,dims))
    for i in range (0, dims):
        for j in range (0, dims):
            if i == j:
                c_matrix[i,j]= TRUE [i]
            else:
                c_matrix[i,j]= FALSE [i]
    if perc == True:
        c_matrix = c_matrix/sum(c_matrix)
    return c_matrix

# function to calculate F1/pre/acc/sen/spe

In [ ]:
def calculate_FPASS (cm,cm_label):
    dims = len(cm_label)
    results=pd.DataFrame(index=range(0,dims+1), columns=['class','accuracy','sensitivity','specificity','precision','F1'])
    results.iloc[:-1,0]=cm_label

    
    for i in range(dims):
        TP = cm[i][i]
        FN = np.sum(cm[i,:])-TP
        FP = np.sum(cm[:,i])-TP
        TN = np.sum(np.sum(cm))-TP-FP-FN
        results.iloc[i,2] = (TP)/(TP+FN)
        results.iloc[i,3] = (TN)/(TN+FP)
        results.iloc[i,4] = (TP)/(TP+FP)
        results.iloc[i,5] = 2*(TP)/(2*TP+FP+FN)
        results.iloc[i,1] = (results.iloc[i,2]+results.iloc[i,3])/2
    
    results.iloc[i+1,0]='Average'
    results.iloc[i+1,1] = np.mean(results.iloc[:,1])
    results.iloc[i+1,2] = np.mean(results.iloc[:,2])
    results.iloc[i+1,3] = np.mean(results.iloc[:,3])
    results.iloc[i+1,4] = np.mean(results.iloc[:,4])
    results.iloc[i+1,5] = np.mean(results.iloc[:,5])
    return results

# function to do ROC

In [ ]:
def do_ROC (X,Y, method = None, cvsplit = 'kfold', n_splits = 10, group = None, plotwhich = 'macro', 
            saved_model = False, saveprobs = False):    
    if method == 'LR':
        if weights != None:
            model = ((LogisticRegression(solver='liblinear',class_weight = weights)))
        else:
            model = ((LogisticRegression(solver='liblinear')))

    if method == 'LDA':
        model = ((LinearDiscriminantAnalysis()))
    if method == 'KNN':
        model = ((KNeighborsClassifier()))
    if method == 'CART':
        model = ((DecisionTreeClassifier()))
    if method == 'NB':
        model = ((GaussianNB()))
    if method == 'SVM':
        if weights != None:
            model = ((SVC(probability=True, kernel = 'linear',
                                      decision_function_shape='ovo',class_weight = weights)))
        else:
            model = ((SVC(probability=True, kernel = 'linear',
                                      decision_function_shape='ovo')))
    if method == 'RF':
        numIterations = 10;
        num_trees = 150
        max_features = 10
        n_jobs = -1
        
        if weights != None:
            model = ((RandomForestClassifier(n_estimators= num_trees, max_features=max_features, 
                                             n_jobs = n_jobs,class_weight = weights)))
        else:
            model = ((RandomForestClassifier(n_estimators= num_trees, max_features=max_features, 
                                             n_jobs = n_jobs)))
    
    if cvsplit == 'loo':
        cv = LeaveOneOut()
    if cvsplit == 'kfold':
        cv = StratifiedKFold(n_splits=n_splits)
    
    cm_label = np.unique(Y)
    if saved_model != False:
        model = saved_model
        all_probs = model.predict_proba(X)
        all_y = Y
    else:
        all_y = []
        all_probs=[]
        if group is not None:
            if cvsplit == 'kfold':
                print('kfold')
                for train, test in cv.split(X, group):
                    all_y.append(Y[test])
                    all_probs.append(model.fit(X[train], Y[train]).predict_proba(X[test]))
            else:
                print('LOGO')
                for train, test in cv.split(X, group):
                    all_y.append(Y[test])
                    all_probs.append(model.fit(X[train], Y[train]).predict_proba(X[test]))
        else:
            for train, test in cv.split(X, Y):
                all_y.append(Y[test])
                all_probs.append(model.fit(X[train], Y[train]).predict_proba(X[test]))

        all_y = np.array(all_y)
        all_probs = np.array(all_probs)

        if cvsplit == 'kfold':
            n = randrange(n_splits)
            all_probs_n = all_probs[n]
            all_y_n = all_y[n]
    #     all_probs_np = []
    #     for i in range(len(X)):
    #         all_probs_np.append(np.array(all_probs[i]))
    n_classes = len(labelnames)
    print('there are '+ str(n_classes)+' classes')


    TPRs = dict()
    FPRs = dict()
    AUCs = dict()
    for i in range(n_classes):
        if cvsplit == 'loo':
            FPRs[i], TPRs[i], _ = roc_curve(all_y,all_probs[:,0,i],pos_label=cm_label[i])
#             FPRs[i], TPRs[i], _ = roc_curve(all_y,all_probs[i],pos_label=cm_label[i])
        if cvsplit == 'kfold':
            FPRs[i], TPRs[i], _ = roc_curve(all_y_n,all_probs_n[:,i],pos_label=cm_label[i])

        AUCs[i] = auc(FPRs[i], TPRs[i])

    # First aggregate all false positive rates
    all_fpr = np.unique(np.concatenate([FPRs[i] for i in range(n_classes)]))

    # Then interpolate all ROC curves at these points
    mean_tpr = np.zeros_like(all_fpr)
    for i in range(n_classes):
        mean_tpr += np.interp(all_fpr, FPRs[i], TPRs[i])

    # Finally average it and compute AUC
    mean_tpr /= n_classes
    mean_tpr[0] = 0.0

    FPRs["macro"] = all_fpr
    TPRs["macro"] = mean_tpr
    AUCs["macro"] = auc(FPRs["macro"], TPRs["macro"])
    
    # plot
    sns.set_context("notebook")
    plt.figure(1, figsize=(10,10))
    if plotwhich == 'macro':
        plt.plot(FPRs[plotwhich], TPRs[plotwhich], lw=4, alpha=0.5, 
                 label='(%s) ROC %s  (AUC = %0.2f)' % (cvsplit, 'macro-averaged', AUCs[plotwhich]))
    else:
        plt.plot(FPRs[plotwhich], TPRs[plotwhich], lw=4, alpha=0.5, 
                 label='(%s) ROC for %s  (AUC = %0.2f)' % (cvsplit, cm_label[plotwhich], AUCs[plotwhich]))
    
    plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='k', alpha=.8)
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
    plt.xlabel('False Positive Rate',fontsize=20)
    plt.ylabel('True Positive Rate',fontsize=20)
    plt.title('Receiver Operating Characteristic (%s)' %method,fontsize=20)
    leg = plt.legend(loc="lower right")
    leg_lines = leg.get_lines()
    leg_texts = leg.get_texts()
    plt.setp(leg_lines, linewidth=4)
    plt.setp(leg_texts, fontsize='x-large')
    plt.grid()
    plt.show()
    
    if saveprobs == True:
        if cvsplit == 'loo':
            probs_out = pd.DataFrame(data=all_probs[:,0,:],columns = labelnames)
        if cvsplit == 'kfold':
            probs_out = pd.DataFrame(data=all_probs,columns = labelnames)
        probs_out.to_csv (r'pred_probs.csv', index = False, header=True)
        
    return model, all_probs, all_y, FPRs, TPRs, AUCs

# function to load data consistently

In [ ]:
def load_data (data_dir = os.getcwd(), features = None, threshold = 100):
    file = pd.read_csv(data_dir)
    columns = file.columns.values
    for i in range (1,len(columns)):
        try:
            float(columns[i])
            n = i
            break
        except:
            continue

    Data = file.iloc[:,n:]
    
    # clean any 'n/a' in data
    mask = Data.isnull()
    Data = Data[~mask]
    
    mz = Data.columns.values 
    mz = mz.astype(float)
    if features is not None:
        print('test data truncated to include only selected spectral features within (%0.2f) ppm' % threshold)
#         indices = np.zeros((1,len(features)))
        indices = [None]*len(features)
        for j in range (0,len(features)):
#             print(j)
            index = np.argmin(abs(mz-features[j]))
            if (mz[index]-features[j])/features[j]*(10^6) <= threshold:
                indices[j] = (index)
        mz = mz[indices]
        new_indices = [x+n for x in indices]
        new_indices = list(range(n))+new_indices
        file = file.iloc[:,new_indices]
        Data = Data.iloc[:,indices]
    else:
        print('data already has the same spectral variables')
        
    return file, Data, mz

# functions for LR feature selection

In [ ]:
def LR_importance (X, Y, num_features, save = True):
#     model = ((LogisticRegression(solver='liblinear',class_weight = weights)))
    model.fit(X, Y)
    importance_LR=np.abs(model.coef_)
    size = importance_LR.shape
    importance_LR=np.reshape(importance_LR,(size[0], size[1]))
    
    LR_features = []
    for i in range(size[0]):
        idx_features = (-importance_LR[i,:]).argsort()[:num_features]
        LR_features.append(mz[idx_features])
    LR_features=np.reshape(LR_features,(num_features, size[0]))
    try:
        features_out=pd.DataFrame(LR_features, columns = model.classes_)
    except:
        features_out=pd.DataFrame(LR_features)
#     print(model.classes_)
    
    if save == True:
        features_out.to_csv (r'LR_features.csv', index = False, header=True)
    return features_out, importance_LR

def deviance(y, yp):
    return 2*log_loss(y, yp, normalize=False)

def LR_loglike(X,Y, threshold = 1.5, save = True):
    GOF=np.zeros((X.shape[1],1))
    chi2_pvals=np.zeros((X.shape[1],1))
    for p in range(X.shape[1]):
        yp=model.fit(X,Y).predict_proba(X)
        yp_mean=np.zeros((len(X),2))
        for i in range (0,len(X)):
            yp_mean[i,0]=yp[:,0].mean()
            yp_mean[i,1]=yp[:,1].mean()
        res_deviance=deviance(Y,yp)
        null_deviance=deviance(Y,yp_mean)
        GOF[p]=null_deviance-res_deviance
    
    chi2_pvals=1-chi2.cdf(GOF , df=1)
#     plt.plot(mz,-np.log10(chi2_pvals))

    idx_features = (-np.log10(chi2_pvals) > threshold)
    mz_dum=mz.reshape(-1,1)
    LR_features2=mz_dum[idx_features]
    
    features_out=pd.DataFrame(LR_features2)
    if save == True:
        features_out.to_csv (r'LR_features2.csv', index = False, header=True)
    return features_out

# functions for RFE feature selection

In [ ]:
def RFE_select (X, Y, n_features, iterate = 1, save = True):
    RFE_features = []
    start_time = time.time()
    for i in range (0,iterate):
#         print(i)
        selector = RFE(model, n_features_to_select=n_features, step=1)
        selector = selector.fit(X, Y)
        RFE_features.append(selector.support_)
        
#     for i, ite in enumerate(RFE_features):
#         Data_r3 = X
#         Data_r3 = Data_r3.T
#         mask = ite
#         Data_r3 = Data_r3[mask]
#         Data_r3 = Data_r3.T
    features_out = mz[tuple(RFE_features)]
    features_out=pd.DataFrame(features_out)
    elapsed_time = time.time() - start_time
    print()
    print('that took '+str(elapsed_time)+' seconds')
    if save == True:
        features_out.to_csv (r'RFE_features1.csv', index = False, header=True)
    return features_out

def RFECV_select (X, Y, iterate = 1, save = True):
    RFECV_features = []
    start_time = time.time()
    for i in range (0,iterate):
#         print(i)
        selector = RFECV(model, step=1, cv = 5, scoring = 'f1_weighted')
        selector = selector.fit(X,Y)
        RFECV_features.append(selector.support_)
        
    features_out = mz[tuple(RFECV_features)]
    features_out=pd.DataFrame(features_out)
    elapsed_time = time.time() - start_time
    print()
    print('that took '+str(elapsed_time)+' seconds')
    if save == True:
        features_out.to_csv (r'RFE_features2.csv', index = False, header=True)
    return features_out

# function for LASSO classification & feature selection

In [ ]:
def LASSO_select(X,Y, max_iter = 10000, tol = 1e-3, n_alphas = 1000, cv=10, save=True):
    target = label_binarize(Y,classes=labelnames)

    lasso=LassoCV(max_iter=max_iter,tol=tol,n_alphas=n_alphas,cv=cv)
    # lasso = Lasso(max_iter=2000,tol=1e-3,)

    clf2=lasso.fit(X,target)
    print('the LASSO classification accuracy was ' +str(clf2.score(X,target)))
    
    importance = np.abs(clf2.coef_)
    selection=(importance != 0)
    lasso_features=mz[selection]
    
    features_out=pd.DataFrame(lasso_features)
    if save == True:
        features_out.to_csv (r'LASSO_features.csv', index = False, header=True)
    return features_out

# function to find overlapping features from different methods

In [ ]:
def match_features(list_of_features,ppm_thresh = 100):
    for n, features in enumerate(list_of_features):
#         print(n)
        if n == 0:
            if isinstance(features, pd.DataFrame):
                features_matched = features.values
            else:
                features_matched = features
        else:
            for feature in features:
                index= np.argmin(abs(features_matched-float(feature)))
                if (features_matched[index]-feature)/feature*(10^6) > 100:
                    np.delete(features_matched, index)
    return features_matched

# function to box plot

In [ ]:
def box_plot (features,mz,box_data,data_label, hue_in = 'no', save = False, savepath = None, swarmyes = True):
    group_label = list(box_data.columns)
    sns.set_style("darkgrid")
    sns.set_context("poster")
#     for i in range(group_label):
    #         colors = {"APC_control": "g", "APC_sim": "g","APC_KRAS_control": "r", "APC_KRAS_sim": "r",
#                  "APC_P53_control": "y", "APC_P53_sim": "y","APC_KRAS_P53_control": "b", "APC_KRAS_P53_sim": "b"}
    for feature in features:
        index= np.argmin(abs(mz-float(feature)))
        fig = plt.figure(figsize=(15,8))
        offset = len(box_data.columns)-len(mz)
        if box_data.shape[0]>10000:
            print('data too large! plotting using randomised 30% data points.')
            sample = np.ceil(box_data.shape[0]/3)
            sample = int(sample)
            box_data = box_data.sample(n=sample, random_state=1)
            swarmyes = False
        if hue_in == 'no':
            sns.boxplot(x=data_label, y=box_data.columns[index+offset], data=box_data, whis=5, palette = 'bright')
            if swarmyes:
                sns.swarmplot(x=data_label, y=box_data.columns[index+offset], data=box_data)
        else:
            sns.boxplot(x=data_label, y=box_data.columns[index+offset], hue=hue_in, data=box_data, whis=5, palette = 'bright')
            if swarmyes:
                sns.swarmplot(x=data_label, y=box_data.columns[index+offset], hue=hue_in, data=box_data)
        if save == True:
            filename = 'BP_'+box_data.columns[index+offset]
            path = os.path.join(savepath, 'box_plots')
            if not os.path.exists(path):
                os.mkdir(path)
            os.chdir(path)
            fig.savefig(filename+'.png')
            plt.close()
            print('plot for '+str(feature)+' saved.')
        else:
            plt.show()
    os.chdir('..')